In [1]:
from transformers import RobertaTokenizerFast, RobertaForQuestionAnswering, DefaultDataCollator, TrainingArguments, Trainer, AutoModelForQuestionAnswering
from datasets import Dataset
import json
import torch


C:\Users\ADMIN\AppData\Roaming\Python\Python37\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Use model 
model = RobertaForQuestionAnswering.from_pretrained("vinai/phobert-base")
tokenizer = RobertaTokenizerFast.from_pretrained("vinai/phobert-base")


def tokenize_function(examples):
    return tokenizer(examples["context"], padding="max_length", truncation=True)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a

In [3]:
# Function read file train
def read_data(filePath):
    f = open(filePath,encoding='utf-8')
    fileRead = json.load(f)
    context = []
    question = []
    answer = []
    for object in fileRead:
        for qa in object["qas"]:
            context.append(object["context"].lower())
            question.append(qa["question"].lower())
            answer.append({
                "text": [qa["answer"]["text"][0].lower()],
                "start": [qa["answer"]["start"][0]]
            })
    dataset = {
    "context": context,
    "question": question,
    "answer": answer
    }
    return dataset

In [83]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=128,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answer"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["start"][0]
        end_char = answer["start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [105]:
dataset = read_data("./QA_data/qa_train.json")
eval_dataset = read_data("./QA_data/qa_eval.json")

In [106]:
dataset_train = Dataset.from_dict(dataset)
dataset_eval = Dataset.from_dict(eval_dataset)

In [107]:
dataset_train

Dataset({
    features: ['context', 'question', 'answer'],
    num_rows: 1119
})

In [108]:
tokenized_squad = dataset_train.map(preprocess_function, batched=True, remove_columns=dataset_train.column_names)
tokenized_squad_eval = dataset_eval.map(preprocess_function, batched=True, remove_columns=dataset_eval.column_names)

In [110]:
tokenized_squad

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 1119
})

In [111]:
data_collator = DefaultDataCollator()

In [115]:
training_args = TrainingArguments(
    output_dir="phobert_law",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    push_to_hub=False,
)

In [116]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad,
    eval_dataset=tokenized_squad_eval,
    tokenizer=tokenizer,
    data_collator=data_collator,
    )

In [117]:
trainer.train()

  1%|          | 70/7000 [16:07<27:14:09, 14.15s/it]













                                                    
                                                 
  1%|          | 70/7000 [16:57<27:14:09, 14.15s/it]


{'eval_loss': 1.005435824394226, 'eval_runtime': 50.3835, 'eval_samples_per_second': 3.672, 'eval_steps_per_second': 0.238, 'epoch': 1.0}


  2%|▏         | 140/7000 [32:53<23:28:57, 12.32s/it]













                                                     
                                                 
  2%|▏         | 140/7000 [33:37<23:28:57, 12.32s/it]


{'eval_loss': 0.7314278483390808, 'eval_runtime': 43.7416, 'eval_samples_per_second': 4.229, 'eval_steps_per_second': 0.274, 'epoch': 2.0}


  3%|▎         | 210/7000 [48:59<25:36:55, 13.58s/it]













                                                     
                                                 
  3%|▎         | 210/7000 [49:47<25:36:55, 13.58s/it]


...

                                                      
100%|██████████| 7000/7000 [26:42:10<00:00, 13.37s/it] 

{'loss': 0.0465, 'learning_rate': 0.0, 'epoch': 100.0}
















                                                      
                                                    
100%|██████████| 7000/7000 [26:43:01<00:00, 13.37s/it]
                                                      
100%|██████████| 7000/7000 [26:43:01<00:00, 13.74s/it] 

{'eval_loss': 0.49616506695747375, 'eval_runtime': 47.5344, 'eval_samples_per_second': 3.892, 'eval_steps_per_second': 0.252, 'epoch': 100.0}
{'train_runtime': 96181.9408, 'train_samples_per_second': 1.163, 'train_steps_per_second': 0.073, 'train_loss': 0.06898140634809222, 'epoch': 100.0}


TrainOutput(global_step=7000, training_loss=0.06898140634809222, metrics={'train_runtime': 96181.9408, 'train_samples_per_second': 1.163, 'train_steps_per_second': 0.073, 'train_loss': 0.06898140634809222, 'epoch': 100.0})

In [118]:
trainer.save_model()

In [126]:
text = "di chúc bằng văn bản có giá trị như di chúc được công chứng hoặc chứng thực: di chúc của quân nhân tại ngũ có xác nhận của thủ trưởng đơn vị từ cấp đại đội trở lên, nếu quân nhân không thể yêu cầu công chứng hoặc chứng thực, di chúc của người đang đi trên tàu biển, máy bay có xác nhận của người chỉ huy phương tiện đó, di chúc của người đang điều trị tại bệnh viện, cơ sở chữa bệnh, điều dưỡng khác có xác nhận của người phụ trách bệnh viện, cơ sở đó, di chúc của người đang làm công việc khảo sát, thăm dò, nghiên cứu ở vùng rừng núi, hải đảo có xác nhận của người phụ trách đơn vị, di chúc của công dân việt nam đang ở nước ngoài có chứng nhận của cơ quan lãnh sự, đại diện ngoại giao việt nam ở nước đó, di chúc của người đang bị tạm giam, tạm giữ, đang chấp hành hình phạt tù, người đang chấp hành biện pháp xử lý hành chính tại cơ sở giáo dục, cơ sở chữa bệnh có xác nhận của người phụ trách cơ sở đó ; điều 638."
question = "Quyền dân sự được công nhận ở đâu ?"
question2= "di chúc của người đang đi trên tàu biển, máy bay có xác nhận của ai ?"

In [120]:
inputs = tokenizer(question2, text, return_tensors="pt")

In [121]:
model = RobertaForQuestionAnswering.from_pretrained("phobert_law")
tokenizer = RobertaTokenizerFast.from_pretrained("phobert_law")



In [125]:
with torch.no_grad():
    outputs = model(**inputs)

RuntimeError: The expanded size of the tensor (261) must match the existing size (258) at non-singleton dimension 1.  Target sizes: [1, 261].  Tensor sizes: [1, 258]

In [68]:
answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

In [69]:
answer_start_index

tensor(248)

In [70]:
answer_end_index

tensor(248)

In [71]:
predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens)

'đó'